# Imports

In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import gensim.downloader as api

# Data Loading

In [ ]:
X_reviews = pd.read_csv("./../data/data_proc/X_proc.csv")

# Bert embedding


In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, Birch, BisectingKMeans
from sentence_transformers import SentenceTransformer # Make sure you have done a "pip install -e ." to have SentenceTransformer package installed

In [ ]:
bert = SentenceTransformer('all-MiniLM-L6-v2') 

In [ ]:
bert_embeddings = bert.encode(X_reviews["txt"]) 
np.shape(bert_embeddings)

# Agglomerative clustering

In [ ]:
N_CLUSTER = 5_000

In [ ]:
clustering_bert = AgglomerativeClustering(n_clusters=N_CLUSTER).fit(bert_embeddings)

In [ ]:
X_reviews["clustering_label_bert"] = clustering_bert.labels_
X_reviews["vector"] = bert_embeddings # TODO doesn't work

# Loading metadata

In [ ]:
metadata_movies = pd.read_json("./../data/raw_data/raw_movies/metadata.json", lines=True)
metadata_books = pd.read_json("./../data/raw_data/raw_book/metadata.json", lines=True)

In [ ]:
metadata_movies.rename({"item_id":"item_id_movie", "title":"movie_title"}, axis='columns',inplace=True)
metadata_books.rename({"item_id":"item_id_book", "title":"book_title"}, axis='columns',inplace=True)

In [ ]:
merged_movies_complete = pd.merge(X_reviews, metadata_movies, on="item_id_movie", how="left")
merged_all_bert = pd.merge(merged_movies_complete, metadata_books, on="item_id_book", how="left")

In [ ]:
clustered = merged_all_bert[["movie_title","book_title","clustering_label_bert","is_movie", "item_id_movie", "item_id_book" ]]
clustered = clustered.fillna("")

# Recommendation